In [1]:
import sympy
import numpy as np

from scipy.interpolate import interp1d
from sympy.abc import x

In [2]:
def prime_generator(lo, hi):
    ''' Generates a random prime number
    
        lo: Low bound for searching
        hi: High bound for searching
        
        Returns: Random prime within bounds
    '''
    
    return 1237 # totally random

def random_distinct(lo, hi, size):
    ''' Used to get lists of distinct random numbers
    
        lo: Low bound for searching
        hi: High bound for searching
        
        Returns: Random numbers within bounds
    '''
    
    v = []
    
    while len(v) < size:
        rand = np.random.randint(lo, hi)
        
        if rand not in v:
            v.append(rand)
            
    return v

In [9]:
def generate_polynomial(data, degree):
    ''' Makes a random polynomial
    
        data: The data as a numeric value to be split
        degree: The degree of the polynomial, and the minimum number of keys needed to decrypt
        
        Returns: polynomial as a function
    '''
    
    # will eventually give random primes greater than max{n, data}
    p = prime_generator(None, None)
    
    # Shamir's algorithms requires each coefficient to be distinct
    coefficients = random_distinct(1, p, degree)
    
    # start building our polynomial
    polynomial = ''
    power = degree
    
    while power > 0:
        # each a_i*x^i for i from 1 to k-1
        polynomial = polynomial + '{}*x**{}+'.format(coefficients[-power], power)
        power -= 1
        
    # set a_0 to data
    polynomial = polynomial + '{}'.format(data)
    
    # restrict the function to finite field
    #polynomial = '({})%{}'.format(polynomial, p)
    
    # convert polynomial string to lambda function
    f = sympy.utilities.lambdify(x, sympy.sympify(polynomial))
    
    return polynomial, f, p

def generate_keys(n, poly, p):
    ''' Creates the keys
    
        n: Number of keys to be created
        poly: Polynomial function to create the keys
        p: Prime number for modular arithmetic
        
        Returns: n-tuple of keys
    '''
    
    # create distinct x values because f(a)=f(b) iff a=b 
    # implies less than n keys will be made
    X = random_distinct(1, p, n)
    
    # get corresponding y values
    Y = [poly(x) for x in X]
    
    return list(zip(X, Y))

def decrypt(keys, p):
    ''' Uses Lagrange interpolation to decrypt data
    
        keys: List of keys to use in the decryption
        p: Prime number used to create finite field
        
        Returns: Decrypted data
    '''
    
    x, y = zip(*keys)
    summ = 0
    
    for i in range(len(y)):   
        product = y[i]
        
        for j in range(len(x)):      
            if i == j:
                continue
                
            product *= x[j]/(x[j]-x[i])

        summ += product
        
    return summ

def decrypt2(keys, p):
    x, y = zip(*keys)
    s = ''
    
    for i in range(len(y)):
        s = s + '{}'.format(y[i])
        
        for j in range(len(x)):
            if i == j:
                continue
                
            s = s + '*{0}/({0}-{1})'.format(x[j], x[i])
        
        if i != len(y) - 1:
            s = s + '+'
            
    return sympy.sympify(s)

In [10]:
s, f, p = generate_polynomial(data=10, degree=2)
s, p

('1169*x**2+1044*x**1+10', 1237)

In [11]:
keys = generate_keys(n=10, poly=f, p=p)
keys

[(1030, 1241267430),
 (945, 1044932815),
 (261, 79905943),
 (594, 413085430),
 (351, 144388423),
 (692, 560514474),
 (736, 634011018),
 (931, 1014215583),
 (442, 228841974),
 (929, 1009864815)]

In [12]:
sample = [keys[0], keys[3], keys[-1]]
sample

[(1030, 1241267430), (594, 413085430), (929, 1009864815)]

In [13]:
decrypt(keys, p) % 1237

10.0